In [1]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant import pipeline
from llm_data_quality_assistant.corruptor import RowCorruptionTypes, CellCorruptionTypes
from llm_data_quality_assistant.enums import Models
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
import numpy as np

load_dotenv()


True

In [2]:
corrupt_dataset = pd.read_csv("../datasets/parker_datasets/allergen/allergen_corrupted_first1000.csv")

gold_standard = pd.read_csv("../datasets/parker_datasets/allergen/allergen_cleaned_gold_first1000.csv")

with open("../datasets/parker_datasets/allergen/allergen.partialkey", "r") as f:
    partial_keys = f.read()

with open("../datasets/parker_datasets/allergen/allergen.rules", "r") as f:
    rules = f.read()


print(partial_keys)
print(rules)
print(corrupt_dataset.head(2))
print(gold_standard.head(2))
print(type(gold_standard.get("code").iloc[0]))
print(type(corrupt_dataset.get("code").iloc[0]))

code ->
    nuts,
    almondnuts,
    brazil_nuts,
    macadamia_nuts,
    hazelnut,
    pistachio,
    walnut,
    cashew,
    celery,
    crustaceans,
    eggs,
    fish,
    gluten,
    lupin,
    milk,
    molluscs,
    mustard,
    peanut,
    sesame,
    soy,
    sulfite

-- Attribute contracts
@nuts:integer
@almondnuts:integer
@brazil_nuts:integer
@macadamia_nuts:integer
@hazelnut:integer
@pistachio:integer
@walnut:integer
@cashew:integer
@celery:integer
@crustaceans:integer
@eggs:integer
@fish:integer
@gluten:integer
@lupin:integer
@milk:integer
@molluscs:integer
@mustard:integer
@peanut:integer
@sesame:integer
@soy:integer
@sulfite:integer


The attributes
of this dataset indicate the presence (‘2’), traces (‘1’), or absence
(‘0’) of allergens in a product.

Everything value except of the code has to be between 0 and 2.

--NOT ALLOWED THAT
nuts < almondnuts
nuts < brazil_nuts
nuts < macadamia_nuts
nuts < hazelnut
nuts < pistachio
nuts < walnut
nuts < cashew

IT ALLWAYS MUST BE

In [3]:
# shortened_corrupt_df = corrupt_dataset[corrupt_dataset["code"].isin(gold_standard["code"])]
# shortened_corrupt_df = shortened_corrupt_df.sort_values(by="code").reset_index(drop=True)

# print("Shape shortened corrupt dataset:")
# print(shortened_corrupt_df.shape)

# print(shortened_corrupt_df)

# shortened_gold_standard = gold_standard[gold_standard["code"].isin(shortened_corrupt_df["code"])]
# shortened_gold_standard = shortened_gold_standard.sort_values(by="code").reset_index(drop=True)

# print("Shape shortened gold standard dataset:")
# print(shortened_gold_standard.shape)

# print(shortened_gold_standard)


In [4]:
# Clean and evaluate using the new Pipeline API
from llm_data_quality_assistant.pipeline import Pipeline
from llm_data_quality_assistant.enums import Models

# Use a primary key for merging
primary_key = "code"
model = Models.GeminiModels.GEMINI_2_0_FLASH_LITE
rpm = 30

# Merge/clean with LLM
merged_df = Pipeline.merge_with_llm(
    dataset=corrupt_dataset,
    primary_key=primary_key,
    model_name=model,
    rpm=rpm,
    additional_prompt=rules,
    verbose=False,
    status_bar=True,
)
# # Show DataFrames for inspection
# print("Merged DataFrame:")
# print(merged_df)
# print("Gold Standard DataFrame:")
# print(shortened_gold_standard)
# print("Corrupted DataFrame:")
# print(shortened_corrupt_df)

Merging groups with LLM:  93%|█████████▎| 96/103 [03:17<00:14,  2.06s/it]


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash-lite'}, 'quotaValue': '1500'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '40s'}]}}

In [ ]:
# Evaluate results
stats_micro = Pipeline.evaluate_micro(
    gold_standard=gold_standard,
    cleaned_dataset=merged_df,
    corrupted_dataset=corrupt_dataset
)
print("====================================")
print("MICRO EVALUATION RESULTS")
print("====================================")
pprint(stats_micro)

stats_macro = Pipeline.evaluate_macro(
    gold_standard=gold_standard,
    cleaned_dataset=merged_df,
    corrupted_dataset=corrupt_dataset
)
print("====================================")
print("MACRO EVALUATION RESULTS")
print("====================================")
pprint(stats_macro)



NameError: name 'Pipeline' is not defined